In [ ]:
# Import library
import requests 
import pandas as pd

from datetime import date
from datetime import timedelta

import datetime
import time

# Create API key list based on all the client:

In [ ]:
client_list = pd.read_csv(r'FILE', index_col=0) 

# Filter clients that have API key
client_list  = client_list[['Client ID', 'Client Type', 'API key', 'Name', 'Market']]
client_list  = client_list[client_list['API key'].notnull()]

# Clean data
client_id    = []
for i in client_list['Client ID']:
    client_id.append(i.split('S', 2)[1])
    
client_list['Client ID'] = client_id

# Rename column
client_list.columns = ['shop_id', 'client_type', 'api_key', 'shop_name', 'country']

# Create client list
client_list.to_csv(r'FILE')

# Make API call and parse into JSON

In [ ]:
thirty_days_ago   = (date.today() + timedelta(days= -30)).strftime("%Y-%m-%d")
yesterday         = (date.today() + timedelta(days= -1 )).strftime("%Y-%m-%d")

a          = datetime.date(int(thirty_days_ago[0:4]), int(thirty_days_ago[5:7]), int(thirty_days_ago[8:10]))
start_time = str(time.mktime(a.timetuple()))[0:-2]


#--------------------------------------------------

# Limit - Number of orders in each API call:
page_size  = 1000
# Offset - Page number of that API call:
page_num   = 1

# Variable to store data:
data_need  = [] 

#--------------------------------------------------

# Loop through each client's API and retreive data:

for shop_id, api_key, shop_name in zip(client_list['shop_id'], client_list['api_key'], client_list['shop_name']):
    
    # Keep looping and get all the data inside!
    print('Client: ' + shop_name)
    while True:
        url   = f'https://web/api/v15/{shop_id}/orders?api_key={api_key}&number={page_num}&size={page_size}&start={start_time}'
        print(url)
        data  = requests.get(url).json()

        # Are there any data left?
        if len(data['data']) != 0:
            
            # If yes, add them to our Variable 'data_need'. And move on to the next page
            data_need.extend(data['data'])
            page_num  = page_num + 1      
            
        else:
            # If not, move to the next API
            page_num  = 1
            print()
            print('------------')
            # (Get out and move to the next API, start from page 1)
            break

# Save it as a big data frame

In [ ]:
# Now we got data_need that contain all the data. Make it a dataframe:
df1 = pd.DataFrame(data_need)
print(len(df1))

# Make a CSV to store raw data
df1.to_csv(r'FILE')

# Save as dataframe file
df1.to_pickle(r'FILE')